<a href="https://colab.research.google.com/github/mayuras7685/Hack4in-Unkils/blob/main/person_count_in_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install supervision
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 kB 12.4 MB/s eta 0:00:00


In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm" -O mall.mp4 && rm -rf /tmp/cookies.txt

--2023-10-06 11:36:39--  https://docs.google.com/uc?export=download&confirm=&id=1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm
Resolving docs.google.com (docs.google.com)... 142.251.2.100, 142.251.2.139, 142.251.2.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/imvolfhui09bi0u9ut7bvjjupvc954tp/1696592175000/04309230031174164349/*/1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm?e=download&uuid=f73c60ac-ee27-4239-99c3-3015aef54841 [following]
--2023-10-06 11:36:41--  https://doc-0c-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/imvolfhui09bi0u9ut7bvjjupvc954tp/1696592175000/04309230031174164349/*/1M3UuH3QNDWGiH0NmGgHtIgXXGDo_nigm?e=download&uuid=f73c60ac-ee27-4239-99c3-3015aef54841
Resolving doc-0c-ag-docs.googleusercontent.com (doc-0c-ag-docs.googleusercontent.com)... 142.251.2.132, 2607:f

In [3]:
import supervision as sv
import cv2
generator = sv.get_video_frames_generator("./mall.mp4")
iterator = iter(generator)
frame = next(iterator)

cv2.imwrite("frame.jpg", frame)

True

In [9]:
import numpy as np
import cv2

MALL_VIDEO_PATH = "/content/mall.mp4"

polygon = np.array([
[1848, 1350],[1848, 1350],[1488, 1902],[3196, 1906],[2456, 1334],[2456, 1334],[2456, 1334],[1848, 1350]
])


In [10]:
video_info = sv.VideoInfo.from_video_path(MALL_VIDEO_PATH)
zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)

box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

In [11]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:00<00:00, 211MB/s]


In [12]:
def process_frame(frame: np.ndarray, _) -> np.ndarray:
   results = model(frame, imgsz=1280)[0]
   detections = sv.Detections.from_yolov8(results)
   detections = detections[detections.class_id == 0]
   zone.trigger(detections=detections)

   box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

   labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
   frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
   frame = zone_annotator.annotate(scene=frame)


   return frame

In [ ]:
sv.process_video(source_path=MALL_VIDEO_PATH, target_path=f"mall-result.mp4", callback=process_frame)